# 04. Model Training

In [49]:
# 01. APRENDIZADO DE MÁQUINA (CLASSIFICAÇÃO PREDITIVA)

import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import LeaveOneOut, cross_val_predict
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import warnings

warnings.filterwarnings("ignore")

print("="*80)
print(" Iniciando avaliação do modelo final (XGBOOST + K=15)")
print("="*80)

# 1. Carregamento e Isolamento da Tarefa
caminho_csv = '../reports/tabela_features_eeg_completa.csv'
df = pd.read_csv(caminho_csv)

SEED = 97
cv_strategy = LeaveOneOut() # LOOCV (N=42)

condicoes_busca = {
    'Face Feliz': ['Face Feliz'],
    'Face Neutra': ['Face Neutra'],
    'Face Raiva': ['Face Raiva']
}

# 2. Pipeline Final
pipeline_classificacao = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', RobustScaler()),
    ('selector', SelectKBest(score_func=f_classif, k=15)),
    ('clf', XGBClassifier(n_estimators=100, max_depth=3, learning_rate=0.1, 
                          eval_metric='logloss', random_state=SEED))
])

# 3. Validação e Extração de Métricas
for nome_condicao, lista_triggers in condicoes_busca.items():
    print(f"\n" + "-"*60)
    print(f"ESTÍMULO ANALISADO: {nome_condicao.upper()}")
    print("-" * 60)
    
    df_f = df[df['Condicao'].isin(lista_triggers)].copy()
    
    if df_f.empty:
        print(f"Atenção: Sem dados para {nome_condicao}.")
        continue

    # Garantia de integridade amostral (N=42)
    df_f = df_f.groupby(['ID', 'Grupo']).mean(numeric_only=True).reset_index()
    print(f"Amostra consolidada: {df_f.shape[0]} sujeitos.\n")

    y = df_f['Grupo'].apply(lambda x: 1 if 'TEA' in x else 0).values
    X = df_f.drop(columns=['ID', 'Grupo', 'Condicao', 'Tipo'], errors='ignore')

    # Predição Cega (LOOCV)
    y_pred = cross_val_predict(pipeline_classificacao, X, y, cv=cv_strategy)

    acc = accuracy_score(y, y_pred)
    cm = confusion_matrix(y, y_pred)

    print(f" ACURÁCIA GLOBAL DO SISTEMA: {acc:.2%}\n")

    print("Matriz de Confusão:")
    print(f"                   Predito Controle (0) | Predito TEA (1)")
    print(f"Real Controle (0) |        {cm[0,0]:02d}           |        {cm[0,1]:02d}")
    print(f"Real TEA (1)      |        {cm[1,0]:02d}           |        {cm[1,1]:02d}\n")

    print("Relatório de Desempenho (Precision, Recall, F1-Score):")
    report = classification_report(y, y_pred, target_names=['Controle', 'TEA'], digits=3)
    print(report)

print("="*80)

 Iniciando avaliação do modelo final (XGBOOST + K=15)

------------------------------------------------------------
ESTÍMULO ANALISADO: FACE FELIZ
------------------------------------------------------------
Amostra consolidada: 42 sujeitos.

 ACURÁCIA GLOBAL DO SISTEMA: 80.95%

Matriz de Confusão:
                   Predito Controle (0) | Predito TEA (1)
Real Controle (0) |        21           |        03
Real TEA (1)      |        05           |        13

Relatório de Desempenho (Precision, Recall, F1-Score):
              precision    recall  f1-score   support

    Controle      0.808     0.875     0.840        24
         TEA      0.812     0.722     0.765        18

    accuracy                          0.810        42
   macro avg      0.810     0.799     0.802        42
weighted avg      0.810     0.810     0.808        42


------------------------------------------------------------
ESTÍMULO ANALISADO: FACE NEUTRA
------------------------------------------------------------
